In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from selenium import webdriver as webdriver
from bs4 import BeautifulSoup as b
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from db_insert import DBHelper as Db
import urllib.request
import time
import os
import re
import sys
from basic import TourInfo
from selenium import webdriver as driver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import warnings

warnings.filterwarnings(action='ignore') 

## Tea 추천 시스템 구축 이유
### 1. 개인적으로 차 마시는 것이 취미(?)아닌 취미
### 2. 커피를 주로 마시는 우리나라에서 차에 대한 소개도 할 수 있을 것 같은 느낌?
### 3. 추천 시스템에 대해서 전문성을 갖고 싶기 때문
### 4. 차 브랜드도 워낙 많고 상품도 많기 때문에 우선은 소규모 데이터로 진행
### 5. 데이터 추출하면서 느낀점..(홈페이지가 너무 조잡하고 보안 문제로 requests 사용불가..)

In [778]:
url_ = 'https://ratetea.com/'
brand_ = 'Teavana'

In [772]:
def get_simple_df_info(df):
  print("dataframe 사이즈: ", df.shape)
  print("\n")
  print("dataframe 정보")
  print(df.info())
  print("\n")
  print("dataframe 간단 통계량")
  print(df.describe())
  print("\n")
  print("dataframe의 몇몇 데이터 샘플")
  print(df.head())

In [ ]:
def get_data_else(url_, brand_):
    # 임시 데이터프레임 생성 
    data = pd.DataFrame(columns = ['Tea Name', 'Brand Name', 'Tea Rating', 'Tea Else', 'contents'])
    
    url = url_
    brand = brand_
    
    print('url_open... {0} Brand를 수집합니다..'.format(brand))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.set_window_size(900, 900)

    
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="details-button"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="proceed-link"]').click()
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[3]/form/input[1]')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(brand)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[3]/form/input[2]').click()
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/a[2]').click()
    time.sleep(2)
    first = driver.find_elements_by_class_name('review_box')
    for li in first:
    # 상품 하나 하나 접근
        data = data.append({'Tea Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a').text,   
                            'Brand Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a:nth-child(2)').text,
                                     'Tea Rating' : li.find_element_by_css_selector('body > div.main > div > div > div > span > span').text,
                                      'Tea Else' : li.find_element_by_css_selector('body > div.main > div > div > div > span:nth-child(6)').text,
                                      'contents' : li.find_element_by_css_selector('body > div.main > div > div > div > p').text
                                      }, ignore_index=True)
    
    

    # 다음 페이지의 xpath, css_selector 모두 변동적이라 최대한 데이터 끌어모으기 위한 쿼리(오류는 패스로 넘기기)
    pageNum = 1

    while pageNum < 12:
        try:
            elements = driver.find_element_by_xpath(f'/html/body/div[3]/p[1]/a[{pageNum}]')
            elements.click()
            time.sleep(1)

            first = driver.find_elements_by_class_name('review_box')
            for li in first:
            # 상품 하나 하나 접근
                data = data.append({'Tea Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a').text,
                                              'Brand Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a:nth-child(2)').text,
                                             'Tea Rating' : li.find_element_by_css_selector('body > div.main > div > div > div > span > span').text,
                                              'Tea Else' : li.find_element_by_css_selector('body > div.main > div > div > div > span:nth-child(6)').text,
                                              'contents' : li.find_element_by_css_selector('body > div.main > div > div > div > p').text
                                              }, ignore_index=True)
                              
                
        except Exception as e:
            pass

        pageNum += 1
        
    data.reset_index(drop=True, inplace=True)

    #간단 데이터 확인
    print('데이터 수집 완료!!')

    
    return data

In [780]:
def get_data_id(url_, brand_):
    # 임시 데이터프레임 생성 
    data = pd.DataFrame(columns = ['ID'])
    
    brand = brand_
    url = url_
    
    print('url_open... {0} Brand를 수집합니다..'.format(brand))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.set_window_size(900, 900)

    
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="details-button"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="proceed-link"]').click()
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[3]/form/input[1]')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(brand)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[3]/form/input[2]').click()
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/a[2]').click()
    time.sleep(2)  
    second = driver.find_elements_by_css_selector('body > div.main > div > div:nth-child(2)')
    for li in second:
    # 상품 하나 하나 접근
        data = data.append({'ID' : li.find_element_by_css_selector('h3 > a').text}, ignore_index=True)
    

    # 다음 페이지의 xpath, css_selector 모두 변동적이라 최대한 데이터 끌어모으기 위한 쿼리(오류는 패스로 넘기기)
    pageNum = 1

    while pageNum < 12:
        try:
            elements = driver.find_element_by_xpath(f'/html/body/div[3]/p[1]/a[{pageNum}]')
            elements.click()
            time.sleep(1)
                
            second1 = driver.find_elements_by_css_selector('body > div.main > div > div:nth-child(2)')
            for li in second1:
            # 상품 하나 하나 접근
                data = data.append({'ID' : li.find_element_by_css_selector('h3 > a').text}, ignore_index=True)               
                
        except Exception as e:
            pass

        pageNum += 1
        
    data.reset_index(drop=True, inplace=True)

    #간단 데이터 확인
    print('데이터 수집 완료!!')

    
    return data

In [782]:
df1 = get_data_else(url_, brand_)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/yechansmacbook/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


url_open... Teavana Brand를 수집합니다..
데이터 수집 완료!!


In [783]:
df2 = get_data_id(url_, brand_)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/yechansmacbook/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


url_open... Teavana Brand를 수집합니다..
데이터 수집 완료!!


In [ ]:
#df3 = get_data_date(url, brand)

In [784]:
#data = pd.DataFrame(columns = ['ID', 'Tea Name', 'Brand Name', 'Tea Rating', 'Tea Else', 'contents']) 

data1 = pd.concat([df1, df2], axis=1)

In [785]:
data = pd.concat([data, data1], axis=0)
data.dropna(axis = 0, inplace=True)
data

,Tea Name,Brand Name,Tea Rating,Tea Else,contents,ID
0,Ultra Spice Chai,Twinings,84 / 100,Aroma: 9/10 Flavor: 4/5 Value: 4/5,The aroma is nice and spicy. I like the flavor...,Brytta Sóþword
1,Earl Grey,Twinings,60 / 100,Aroma: 6/10 Flavor: 3/5 Value: 4/5,I’m not an expert on earl gray. This may be th...,Bailz
2,Earl Grey,Twinings,40 / 100,Aroma: 5/10 Flavor: 3/5 Value: 3/5,"Cup at home, made from a teabag. Pours hazy da...",Mr Bamsen
3,English Breakfast Lemon,Twinings,48 / 100,Aroma: 5/10 Flavor: 3/5 Value: 3/5,"Dry or steeping, this tea smells of lemon cand...",Difflugia
4,Lady Grey,Twinings,60 / 100,Aroma: 6/10 Flavor: 3/5 Value: 3/5,"Cup at home, made from a teabag. Pours nearly ...",Mr Bamsen
...,...,...,...,...,...,...
83,Blueberry Kona Pop,Teavana,60 / 100,Aroma: 6/10 Flavor: 4/5 Value: 2/5,"Another well-executed fruit blend, albeit over...",Todd
84,Jasmine Dragon Phoenix Pearls Green Tea,Teavana,77 / 100,Aroma: 8/10 Flavor: 4/5 Value: 3/5,"I thought this tea was great, I really enjoyed...",Dominika Oliver
85,Maharaja Chai Oolong Tea,Teavana,87 / 100,Aroma: 10/10 Flavor: 4/5 Value: 3/5,Spiced scented tea mixed with samurai chai mat...,Jet-Tian
86,Spiced Apple Cider Rooibos,Teavana,50 / 100,Aroma: 5/10 Flavor: 3/5 Value: 2/5,I like Teavana Tea. I just think it tends to b...,Lori


In [ ]:
total_tea_data.to_csv('total_tea_data.csv', index = True, encoding='utf-8')

### 추가적인 리뷰 크롤링 작업 (웹 페이지마다 주소도 바뀌고 html도 달라지기 때문에 일일이 크롤링하는 수밖에 없을 듯)

In [818]:
def get_review_data_id(url_):
    # 임시 데이터프레임 생성 
    data = pd.DataFrame(columns = ['ID'])
    
    url = url_
    
    print('url_open... {0} Brand를 수집합니다..'.format('review'))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.set_window_size(900, 900)

    
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="details-button"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="proceed-link"]').click()
    time.sleep(2)  
    second = driver.find_elements_by_css_selector('body > div.main > div:nth-child(2) > div:nth-child(2)')
    for li in second:
    # 상품 하나 하나 접근
        data = data.append({'ID' : li.find_element_by_css_selector('h1').text}, ignore_index=True)
    

    # 다음 페이지의 xpath, css_selector 모두 변동적이라 최대한 데이터 끌어모으기 위한 쿼리(오류는 패스로 넘기기)
    pageNum = 1

    while pageNum < 100:
        try:
            elements = driver.find_element_by_xpath(f'/html/body/div[3]/p[1]/a[{pageNum}]')
            elements.click()
            time.sleep(1)
                
            second1 = driver.find_elements_by_css_selector('body > div.main > div:nth-child(2) > div:nth-child(2)')
            for li in second1:
            # 상품 하나 하나 접근
                data = data.append({'ID' : li.find_element_by_css_selector('h1').text}, ignore_index=True)               
                
        except Exception as e:
            pass

        pageNum += 1
        
    data.reset_index(drop=True, inplace=True)

    #간단 데이터 확인
    print('데이터 수집 완료!!')

    
    return data

### 데이터 수집이 너무 어렵다..

In [1184]:
url_ = 'https://ratetea.com/browse_reviews.php?reviewer_id=1437'
name_ = 'S B Walton'

In [1185]:
def get_review_data(url_, name_):
    # 임시 데이터프레임 생성 
    data = pd.DataFrame(columns = ['ID', 'Tea Name', 'Brand Name', 'Tea Rating', 'Tea Else', 'contents'])
    
    url = url_
    name = name_
    
    print('url_open... {0} Brand를 수집합니다..'.format('review'))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.set_window_size(900, 900)

    
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="details-button"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="proceed-link"]').click()
    time.sleep(2)
    first = driver.find_elements_by_class_name('review_box')
    for li in first:
    # 상품 하나 하나 접근
        data = data.append({'ID' : name,
                            'Tea Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a').text,   
                            'Brand Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a:nth-child(2)').text,
                             'Tea Rating' : li.find_element_by_css_selector('body > div.main > div > div > div > span > span').text,
                              'Tea Else' : li.find_element_by_css_selector('body > div.main > div > div > div > span:nth-child(6)').text,
                              'contents' : li.find_element_by_css_selector('body > div.main > div > div > div > p').text
                                      }, ignore_index=True)
    
    

    # 다음 페이지의 xpath, css_selector 모두 변동적이라 최대한 데이터 끌어모으기 위한 쿼리(오류는 패스로 넘기기)
    pageNum = 1

    while pageNum < 100:
        try:
            elements = driver.find_element_by_xpath(f'/html/body/div[3]/p[1]/a[{pageNum}]')
            elements.click()
            time.sleep(1)

            first = driver.find_elements_by_class_name('review_box')
            for li in first:
            # 상품 하나 하나 접근
                data = data.append({'ID' : name,
                                'Tea Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a').text,
                                      'Brand Name' : li.find_element_by_css_selector('body > div.main > div > div > div > h3 > a:nth-child(2)').text,
                                     'Tea Rating' : li.find_element_by_css_selector('body > div.main > div > div > div > span > span').text,
                                      'Tea Else' : li.find_element_by_css_selector('body > div.main > div > div > div > span:nth-child(6)').text,
                                      'contents' : li.find_element_by_css_selector('body > div.main > div > div > div > p').text
                                              }, ignore_index=True)
                              
                
        except Exception as e:
            pass

        pageNum += 1
        
    data.reset_index(drop=True, inplace=True)

    #간단 데이터 확인
    print('데이터 수집 완료!!')

    
    return data

In [1186]:
df1 = get_review_data(url_, name_)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/yechansmacbook/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


url_open... review Brand를 수집합니다..
데이터 수집 완료!!


In [1187]:
data = pd.concat([data, df1], axis=0)
data

,Tea Name,Brand Name,Tea Rating,Tea Else,contents,ID
0,Ultra Spice Chai,Twinings,84 / 100,Aroma: 9/10 Flavor: 4/5 Value: 4/5,The aroma is nice and spicy. I like the flavor...,Brytta Sóþword
1,Earl Grey,Twinings,60 / 100,Aroma: 6/10 Flavor: 3/5 Value: 4/5,I’m not an expert on earl gray. This may be th...,Bailz
2,Earl Grey,Twinings,40 / 100,Aroma: 5/10 Flavor: 3/5 Value: 3/5,"Cup at home, made from a teabag. Pours hazy da...",Mr Bamsen
3,English Breakfast Lemon,Twinings,48 / 100,Aroma: 5/10 Flavor: 3/5 Value: 3/5,"Dry or steeping, this tea smells of lemon cand...",Difflugia
4,Lady Grey,Twinings,60 / 100,Aroma: 6/10 Flavor: 3/5 Value: 3/5,"Cup at home, made from a teabag. Pours nearly ...",Mr Bamsen
...,...,...,...,...,...,...
1,Gold Blend,Lyons,57 / 100,Aroma: 6/10 Flavor: 3/5 Value: 3/5,This tea was a bit bitter and too strong for m...,S B Walton
2,English Breakfast,Dilmah,83 / 100,Aroma: 7/10 Flavor: 5/5 Value: 5/5,"This is the only single estate, pure Ceylon th...",S B Walton
3,PG Tips Pyramid bags,PG Tips,60 / 100,Aroma: 4/10 Flavor: 3/5 Value: 5/5,PG Tips is really meant to be drunk British st...,S B Walton
4,Gold Blend,Barry's Tea,77 / 100,Aroma: 7/10 Flavor: 4/5 Value: 5/5,"Very good tea, in the Irish style, with a bit ...",S B Walton


In [1188]:
data.to_csv('tea_data_id.csv', encoding='utf-8')